In [36]:
from src.optimisers.hadamard_gate_reduction import Hadamard_gate_reduction
from src.optimisers.single_qubit_gate_cancellation import Single_qubit_gate_cancellation
from src.optimisers.phase_polynomial_optimizer import pretreatment_phase_polynomial

from src.converters.netlist_converters import qCircuit_to_netlist, netlist_to_qCircuit
from src.converters.graph_converter import create_graph_from_netlist, create_netlist_from_graph
from src.converters.phase_polynomial_converter import create_phase_polynomial_from_netlist, create_netlist_from_phase_polynomial

from src.utils.graphs import plot_graph
from mpqp import QCircuit
from mpqp.gates import X, CNOT, Z, H , Rz

In [37]:
circ = QCircuit(2)
    
circ.add(Rz(1, 1))
circ.add(Rz(1, 1))
circ.add(CNOT(0, 1))
circ.add(Rz(2, 1))
circ.add(CNOT(1, 0))
circ.add(CNOT(0, 1)) 
circ.add(Rz(2.5, 1))
circ.add(CNOT(0, 1))
circ.add(Rz(3, 0))
circ.add(Rz(4, 1))
circ.add(CNOT(1, 0))

print(circ)

                                     ┌───┐                     ┌───────┐┌───┐
q_0: ────────────────────■───────────┤ X ├──■───────────────■──┤ Rz(3) ├┤ X ├
     ┌───────┐┌───────┐┌─┴─┐┌───────┐└─┬─┘┌─┴─┐┌─────────┐┌─┴─┐├───────┤└─┬─┘
q_1: ┤ Rz(1) ├┤ Rz(1) ├┤ X ├┤ Rz(2) ├──■──┤ X ├┤ Rz(2.5) ├┤ X ├┤ Rz(4) ├──■──
     └───────┘└───────┘└───┘└───────┘     └───┘└─────────┘└───┘└───────┘     


In [38]:
netlist = qCircuit_to_netlist(circ)
print(netlist)
phase_poly = create_phase_polynomial_from_netlist(netlist)


phase_poly = pretreatment_phase_polynomial(phase_poly)

netlist = create_netlist_from_phase_polynomial(phase_poly)
circ = netlist_to_qCircuit(netlist)
print(circ)

[Rz([1]), Rz([1]), CNOT(·[0], [1]), Rz([1]), CNOT(·[1], [0]), CNOT(·[0], [1]), Rz([1]), CNOT(·[0], [1]), Rz([0]), Rz([1]), CNOT(·[1], [0])]
                   ┌───┐                     ┌───────┐┌───┐
q_0: ───────────■──┤ X ├──■───────────────■──┤ Rz(3) ├┤ X ├
     ┌───────┐┌─┴─┐└─┬─┘┌─┴─┐┌─────────┐┌─┴─┐├───────┤└─┬─┘
q_1: ┤ Rz(2) ├┤ X ├──■──┤ X ├┤ Rz(2.5) ├┤ X ├┤ Rz(6) ├──■──
     └───────┘└───┘     └───┘└─────────┘└───┘└───────┘     


In [39]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
import numpy as np

# Create a complex circuit with Rz and CNOT gates
qc_complex = QuantumCircuit(2)
qc_complex.rz(1, 1)
qc_complex.rz(1, 1)

qc_complex.cx(0, 1)
qc_complex.rz(2, 1)
qc_complex.cx(1, 0)

qc_complex.cx(0, 1)

qc_complex.rz(2.5, 1)
qc_complex.cx(0, 1)


qc_complex.rz(4, 1)
qc_complex.rz(3, 0)
qc_complex.cx(1, 0)


qc_after = QuantumCircuit(2)
qc_after.rz(2, 1)
qc_after.cx(0, 1)
qc_after.cx(1, 0)
qc_after.cx(0, 1)
qc_after.rz(2.5, 1)
qc_after.cx(0, 1)
qc_after.rz(3, 0)
qc_after.rz(6, 1)
qc_after.cx(1, 0)

print(qc_complex)
print(qc_after)


def are_equivalent_up_to_phase(circuit1, circuit2, tolerance=1e-10):
    # Calculate the unitary matrices for both circuits
    unitary1 = Operator(circuit1).data
    unitary2 = Operator(circuit2).data
    
    # Check if dimensions match
    if unitary1.shape != unitary2.shape:
        return False
    
    # Compute the relative phase between the matrices
    product = np.dot(np.conjugate(np.transpose(unitary1)), unitary2)
    trace = np.trace(product)
    phase = trace / unitary1.shape[0]  # Average phase factor
    
    # Verify that the product of U1 and U2† is a scaled identity matrix
    identity_check = product / phase  # Should be close to identity
    
    # Check if identity_check is close to an identity matrix within tolerance
    is_identity = np.allclose(identity_check, np.eye(unitary1.shape[0]), atol=tolerance)
    is_phase_unit = np.isclose(np.abs(phase), 1.0, atol=tolerance)
    
    return is_identity and is_phase_unit

# Check if the circuits are equivalent up to a global phase
equivalent = are_equivalent_up_to_phase(qc_complex, qc_after)
print("The circuits are equivalent up to a global phase:", equivalent)



                                     ┌───┐                     ┌───────┐┌───┐
q_0: ────────────────────■───────────┤ X ├──■───────────────■──┤ Rz(3) ├┤ X ├
     ┌───────┐┌───────┐┌─┴─┐┌───────┐└─┬─┘┌─┴─┐┌─────────┐┌─┴─┐├───────┤└─┬─┘
q_1: ┤ Rz(1) ├┤ Rz(1) ├┤ X ├┤ Rz(2) ├──■──┤ X ├┤ Rz(2.5) ├┤ X ├┤ Rz(4) ├──■──
     └───────┘└───────┘└───┘└───────┘     └───┘└─────────┘└───┘└───────┘     
                   ┌───┐                     ┌───────┐┌───┐
q_0: ───────────■──┤ X ├──■───────────────■──┤ Rz(3) ├┤ X ├
     ┌───────┐┌─┴─┐└─┬─┘┌─┴─┐┌─────────┐┌─┴─┐├───────┤└─┬─┘
q_1: ┤ Rz(2) ├┤ X ├──■──┤ X ├┤ Rz(2.5) ├┤ X ├┤ Rz(6) ├──■──
     └───────┘└───┘     └───┘└─────────┘└───┘└───────┘     
The circuits are equivalent up to a global phase: True


In [40]:
circ = QCircuit(2)

# Define a complex circuit with Rz and CNOT gates
circ = QCircuit(2)

# Define a complex circuit with Rz and CNOT gates
circ.add(Rz(0.5, 1))
circ.add(CNOT(0, 1))
circ.add(Rz(1.2, 1))
circ.add(CNOT(1, 0))
circ.add(Rz(0.8, 0))
circ.add(CNOT(0, 1))
circ.add(Rz(2.3, 1))
circ.add(CNOT(1, 0))
circ.add(Rz(1.1, 1))
circ.add(CNOT(0, 1))
circ.add(Rz(3.4, 1))
circ.add(CNOT(1, 0))
circ.add(CNOT(0, 1))  # Pair of CNOTs with the same control and target, which should cancel out
circ.add(CNOT(0, 1))  # Another redundant pair
circ.add(Rz(0.9, 1))

print(circ)


                                ┌───┐┌─────────┐                ┌───┐»
q_0: ─────────────■─────────────┤ X ├┤ Rz(0.8) ├──■─────────────┤ X ├»
     ┌─────────┐┌─┴─┐┌─────────┐└─┬─┘└─────────┘┌─┴─┐┌─────────┐└─┬─┘»
q_1: ┤ Rz(0.5) ├┤ X ├┤ Rz(1.2) ├──■─────────────┤ X ├┤ Rz(2.3) ├──■──»
     └─────────┘└───┘└─────────┘                └───┘└─────────┘     »
«                                ┌───┐                     
«q_0: ─────────────■─────────────┤ X ├──■────■─────────────
«     ┌─────────┐┌─┴─┐┌─────────┐└─┬─┘┌─┴─┐┌─┴─┐┌─────────┐
«q_1: ┤ Rz(1.1) ├┤ X ├┤ Rz(3.4) ├──■──┤ X ├┤ X ├┤ Rz(0.9) ├
«     └─────────┘└───┘└─────────┘     └───┘└───┘└─────────┘


In [41]:
netlist = qCircuit_to_netlist(circ)
print(netlist)
phase_poly = create_phase_polynomial_from_netlist(netlist)


phase_poly = pretreatment_phase_polynomial(phase_poly)

netlist = create_netlist_from_phase_polynomial(phase_poly)
circ = netlist_to_qCircuit(netlist)
print(circ)

[Rz([1]), CNOT(·[0], [1]), Rz([1]), CNOT(·[1], [0]), Rz([0]), CNOT(·[0], [1]), Rz([1]), CNOT(·[1], [0]), Rz([1]), CNOT(·[0], [1]), Rz([1]), CNOT(·[1], [0]), CNOT(·[0], [1]), CNOT(·[0], [1]), Rz([1])]
                                ┌───┐┌─────────┐                ┌───┐     ┌───┐»
q_0: ─────────────■─────────────┤ X ├┤ Rz(0.8) ├──■─────────────┤ X ├──■──┤ X ├»
     ┌─────────┐┌─┴─┐┌─────────┐└─┬─┘└─────────┘┌─┴─┐┌─────────┐└─┬─┘┌─┴─┐└─┬─┘»
q_1: ┤ Rz(0.5) ├┤ X ├┤ Rz(1.2) ├──■─────────────┤ X ├┤ Rz(3.4) ├──■──┤ X ├──■──»
     └─────────┘└───┘└─────────┘                └───┘└─────────┘     └───┘     »
«                
«q_0: ───────────
«     ┌─────────┐
«q_1: ┤ Rz(4.3) ├
«     └─────────┘


In [42]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
import numpy as np

# Create a complex circuit with Rz and CNOT gates
qc_complex = QuantumCircuit(2)
qc_complex.rz(0.5, 1)
qc_complex.cx(0, 1)
qc_complex.rz(1.2, 1)
qc_complex.cx(1, 0)
qc_complex.rz(0.8, 0)
qc_complex.cx(0, 1)

qc_complex.rz(2.3, 1)
qc_complex.cx(1, 0)
qc_complex.rz(1.1, 1)
qc_complex.cx(0, 1)
qc_complex.rz(3.4, 1)
qc_complex.cx(1, 0)
qc_complex.cx(0, 1)  # Pair of CNOTs with the same control and target, which should cancel out
qc_complex.cx(0, 1)  # Another redundant pair
qc_complex.rz(0.9, 1)



qc_after = QuantumCircuit(2)
qc_after.rz(0.5, 1)
qc_after.cx(0, 1)
qc_after.rz(1.2, 1)
qc_after.cx(1, 0)
qc_after.rz(0.8, 0)
qc_after.cx(0, 1)

qc_after.rz(3.4, 1)
qc_after.cx(1, 0)
qc_after.cx(0, 1)
qc_after.cx(1, 0)
qc_after.rz(4.3, 1)

print(qc_complex)
print(qc_after)

def are_equivalent_up_to_phase(circuit1, circuit2, tolerance=1e-10):
    # Calculate the unitary matrices for both circuits
    unitary1 = Operator(circuit1).data
    unitary2 = Operator(circuit2).data
    
    # Check if dimensions match
    if unitary1.shape != unitary2.shape:
        return False
    
    # Compute the relative phase between the matrices
    product = np.dot(np.conjugate(np.transpose(unitary1)), unitary2)
    trace = np.trace(product)
    phase = trace / unitary1.shape[0]  # Average phase factor
    
    # Verify that the product of U1 and U2† is a scaled identity matrix
    identity_check = product / phase  # Should be close to identity
    
    # Check if identity_check is close to an identity matrix within tolerance
    is_identity = np.allclose(identity_check, np.eye(unitary1.shape[0]), atol=tolerance)
    is_phase_unit = np.isclose(np.abs(phase), 1.0, atol=tolerance)
    
    return is_identity and is_phase_unit

# Check if the circuits are equivalent up to a global phase
equivalent = are_equivalent_up_to_phase(qc_complex, qc_after)
print("The circuits are equivalent up to a global phase:", equivalent)



                                ┌───┐┌─────────┐                ┌───┐»
q_0: ─────────────■─────────────┤ X ├┤ Rz(0.8) ├──■─────────────┤ X ├»
     ┌─────────┐┌─┴─┐┌─────────┐└─┬─┘└─────────┘┌─┴─┐┌─────────┐└─┬─┘»
q_1: ┤ Rz(0.5) ├┤ X ├┤ Rz(1.2) ├──■─────────────┤ X ├┤ Rz(2.3) ├──■──»
     └─────────┘└───┘└─────────┘                └───┘└─────────┘     »
«                                ┌───┐                     
«q_0: ─────────────■─────────────┤ X ├──■────■─────────────
«     ┌─────────┐┌─┴─┐┌─────────┐└─┬─┘┌─┴─┐┌─┴─┐┌─────────┐
«q_1: ┤ Rz(1.1) ├┤ X ├┤ Rz(3.4) ├──■──┤ X ├┤ X ├┤ Rz(0.9) ├
«     └─────────┘└───┘└─────────┘     └───┘└───┘└─────────┘
                                ┌───┐┌─────────┐                ┌───┐     ┌───┐»
q_0: ─────────────■─────────────┤ X ├┤ Rz(0.8) ├──■─────────────┤ X ├──■──┤ X ├»
     ┌─────────┐┌─┴─┐┌─────────┐└─┬─┘└─────────┘┌─┴─┐┌─────────┐└─┬─┘┌─┴─┐└─┬─┘»
q_1: ┤ Rz(0.5) ├┤ X ├┤ Rz(1.2) ├──■─────────────┤ X ├┤ Rz(3.4) ├──■──┤ X ├──■──»
     └─────────┘└───┘

In [43]:
circ = QCircuit(3)
    
circ.add(Rz(1, 1))
circ.add(CNOT(0, 1))
circ.add(CNOT(0, 2))
circ.add(CNOT(1, 0))
circ.add(Rz(1, 0))
circ.add(Rz(2, 1))
circ.add(CNOT(0, 1)) 
circ.add(Rz(2.5, 1))
circ.add(CNOT(0, 1))
circ.add(Rz(3, 0))
circ.add(Rz(4, 1))
circ.add(CNOT(1, 0))

print(circ)

                        ┌───┐┌───────┐                     ┌───────┐┌───┐
q_0: ───────────■────■──┤ X ├┤ Rz(1) ├──■───────────────■──┤ Rz(3) ├┤ X ├
     ┌───────┐┌─┴─┐  │  └─┬─┘├───────┤┌─┴─┐┌─────────┐┌─┴─┐├───────┤└─┬─┘
q_1: ┤ Rz(1) ├┤ X ├──┼────■──┤ Rz(2) ├┤ X ├┤ Rz(2.5) ├┤ X ├┤ Rz(4) ├──■──
     └───────┘└───┘┌─┴─┐     └───────┘└───┘└─────────┘└───┘└───────┘     
q_2: ──────────────┤ X ├─────────────────────────────────────────────────
                   └───┘                                                 


In [44]:
netlist = qCircuit_to_netlist(circ)
print(netlist)
phase_poly = create_phase_polynomial_from_netlist(netlist)


phase_poly = pretreatment_phase_polynomial(phase_poly)

netlist = create_netlist_from_phase_polynomial(phase_poly)
circ = netlist_to_qCircuit(netlist)
print(circ)

[Rz([1]), CNOT(·[0], [1]), CNOT(·[0], [2]), CNOT(·[1], [0]), Rz([0]), Rz([1]), CNOT(·[0], [1]), Rz([1]), CNOT(·[0], [1]), Rz([0]), Rz([1]), CNOT(·[1], [0])]
                        ┌───┐┌───────┐                              ┌───┐
q_0: ───────────■────■──┤ X ├┤ Rz(4) ├──■───────────────■───────────┤ X ├
     ┌───────┐┌─┴─┐  │  └─┬─┘└───────┘┌─┴─┐┌─────────┐┌─┴─┐┌───────┐└─┬─┘
q_1: ┤ Rz(1) ├┤ X ├──┼────■───────────┤ X ├┤ Rz(2.5) ├┤ X ├┤ Rz(6) ├──■──
     └───────┘└───┘┌─┴─┐              └───┘└─────────┘└───┘└───────┘     
q_2: ──────────────┤ X ├─────────────────────────────────────────────────
                   └───┘                                                 


In [45]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
import numpy as np

# Create a complex circuit with Rz and CNOT gates
qc_complex = QuantumCircuit(3)
qc_complex.cx(0, 1)
qc_complex.cx(0, 2)
qc_complex.cx(0, 1)

qc_after = QuantumCircuit(3)
qc_after.cx(0, 2)

print(qc_complex)
print(qc_after)


def are_equivalent_up_to_phase(circuit1, circuit2, tolerance=1e-10):
    # Calculate the unitary matrices for both circuits
    unitary1 = Operator(circuit1).data
    unitary2 = Operator(circuit2).data
    
    # Check if dimensions match
    if unitary1.shape != unitary2.shape:
        return False
    
    # Compute the relative phase between the matrices
    product = np.dot(np.conjugate(np.transpose(unitary1)), unitary2)
    trace = np.trace(product)
    phase = trace / unitary1.shape[0]  # Average phase factor
    
    # Verify that the product of U1 and U2† is a scaled identity matrix
    identity_check = product / phase  # Should be close to identity
    
    # Check if identity_check is close to an identity matrix within tolerance
    is_identity = np.allclose(identity_check, np.eye(unitary1.shape[0]), atol=tolerance)
    is_phase_unit = np.isclose(np.abs(phase), 1.0, atol=tolerance)
    
    return is_identity and is_phase_unit

# Check if the circuits are equivalent up to a global phase
equivalent = are_equivalent_up_to_phase(qc_complex, qc_after)
print("The circuits are equivalent up to a global phase:", equivalent)

                    
q_0: ──■────■────■──
     ┌─┴─┐  │  ┌─┴─┐
q_1: ┤ X ├──┼──┤ X ├
     └───┘┌─┴─┐└───┘
q_2: ─────┤ X ├─────
          └───┘     
          
q_0: ──■──
       │  
q_1: ──┼──
     ┌─┴─┐
q_2: ┤ X ├
     └───┘
The circuits are equivalent up to a global phase: True
